In [11]:
from sklearn.model_selection import train_test_split
import optuna
import sklearn.datasets
import mlflow
import mlflow.sklearn
mlflow.end_run()
mlflow.set_experiment("optuna_experiment")


dataset = sklearn.datasets.load_iris()

X = dataset.data
y = dataset.target

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

def objective(trial):
    iter = trial.suggest_int("max_iter", 1, 100)
        # Train a logistic regression model with the given hyperparameters
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(max_iter=iter)
    model.fit(x_train, y_train)

    # Evaluate the model on the test set
    from sklearn.metrics import accuracy_score
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)

    with mlflow.start_run():
        mlflow.log_params(trial.params)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_param("test_size", 0.8)
        mlflow.sklearn.log_model(model, "model")

    return accuracy


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best trial
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2024-11-03 17:06:53,790] A new study created in memory with name: no-name-90322cb9-7523-445a-887d-4f823024e15d


c:\Users\k\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/11/03 17:07:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\k\AppData\Local\Temp\tmpmqvx55sy\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.4.2', 'cloudpickle==3.1.0']. Set logging level to DEBUG to see the full traceback. 
2024/11/03 17:07:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` pa

Best trial:
  Value: 0.9666666666666667
  Params:
    max_iter: 58
